In [1]:
import re
def load_instructions(limits = None):
    with open('input22.txt') as txtfile:
        instructions = [[line[:3].strip()] + [int(n) for n in re.findall('(-?\d+)', line)] for line in txtfile.readlines()]
        instructions = [[instruction[0]] + [range(instruction[i], instruction[i+1]+1) for i in range(1, len(instruction), 2)] for instruction in instructions]
        if limits: instructions = [[instruction[0]] + cube_intersection(instruction[1:], [limits for _ in range(3)]) for instruction in instructions]

    return instructions

In [2]:
def one_axis_intervals(cube1_axis, cube2_axis):
    def interval(a, b): return range(a, b)

    axis_intersection = cube_axis_intersection(cube1_axis, cube2_axis)
    
    if axis_intersection:
        return [interval(cube1_axis[0], axis_intersection[0]), \
                interval(axis_intersection[0], axis_intersection[-1]+1), \
                interval(axis_intersection[-1]+1, cube1_axis[-1]+1)]
    else:
        return []

def cube_axis_intersection(cube1_axis, cube2_axis):
    if cube1_axis and cube2_axis:
        return range(max(cube1_axis[0], cube2_axis[0]), min(cube1_axis[-1], cube2_axis[-1])+1)
    else:
        return range(0,0)

In [3]:
def cube(instruction):
    return instruction[1:]

def does_exist(cube):
    return all(cube_axis for cube_axis in cube)

def cube_intersection(cube1, cube2):
    return [cube_axis_intersection(cube1_axis, cube2_axis) for cube1_axis, cube2_axis in zip(cube1, cube2)]

def equal_cubes(cube1, cube2):
    return all(all(cube1[i][j] == cube2[i][j] for j in [0,-1]) for i in range(len(cube1)))

def cube_size(cube):
    if not does_exist(cube): return 0
    product = 1
    for cube_axis in cube: product *= (1 + cube_axis[-1] - cube_axis[0])
    return product

def sum_disjoint_cube_sizes(instructions):
    return sum([cube_size(cube(instruction)) for instruction in instructions])

def explode_cube(victim_cube, tnt_cube):
    cubes = []
    cubes_intersection = cube_intersection(victim_cube, tnt_cube)
    if does_exist(cubes_intersection):
        for range_x in one_axis_intervals(victim_cube[0], tnt_cube[0]):
            for range_y in one_axis_intervals(victim_cube[1], tnt_cube[1]):
                for range_z in one_axis_intervals(victim_cube[2], tnt_cube[2]):
                    created_cube = [range_x, range_y, range_z]
                    if does_exist(created_cube) and not equal_cubes(created_cube, cubes_intersection):
                        cubes.append(created_cube)
    else:
        cubes.append(victim_cube)
    return cubes

In [4]:
def next_off_instruction(instructions):
    for idx, instruction in enumerate(instructions):
        if instruction[0] == 'off':
            return idx
    return None

In [5]:
def create_tnt_cubes(instructions):
    next_instructions = []
    while (instructions):
        instruction = instructions.pop(0)
        if instruction[0] == 'off': next_instructions.append(instruction)
        else:
            for next_instruction in next_instructions:
                if next_instruction[0] == 'on':
                    tnt_cube = cube_intersection(cube(instruction), cube(next_instruction))
                    if does_exist(tnt_cube): next_instructions.append(['off'] + tnt_cube)
            next_instructions.append(instruction)

    return next_instructions

In [6]:
def apply_tnt_cubes(instructions):
    first_off_index = next_off_instruction(instructions)
    while (first_off_index is not None):
        disjoint_cubes_instructions = []

        bomb_cube = cube(instructions[first_off_index])
        for i in range(first_off_index):
            disjoint_cubes_instructions.extend([['on'] + cube for cube in explode_cube(cube(instructions[i]), bomb_cube)])

        instructions = disjoint_cubes_instructions + instructions[first_off_index+1:]
        first_off_index = next_off_instruction(instructions)
    return instructions

Part 01

In [7]:
disjoint_cube_sizes =  sum_disjoint_cube_sizes(apply_tnt_cubes(create_tnt_cubes(load_instructions(range(-50, 51)))))
disjoint_cube_sizes

615869

Part 02

In [8]:
disjoint_cube_sizes =  sum_disjoint_cube_sizes(apply_tnt_cubes(create_tnt_cubes(load_instructions())))
disjoint_cube_sizes

1323862415207825